In [4]:
#Fuzzy Logic Car Controller - Cruise Control
#Facts: Speed limits, road types, slopes.
import time
import random
import math
#We create some variables here that we will use in the cruise control. Speed limits are hard coded here.
speedLimits = [50,60,70,80,90,100]
#We create an empty roadType list that will hold the road types we create.
roadType = []
#Here we choose an initial speed limit using a random choice function provided by the random library.
currentLimit = random.choice(speedLimits)
#The car should start at 0 as it is not already driving. However this can be changed to demonstrate a case where it is already driving.
carSpeed = 0
#This randomness variable is used to determine the frequency of changing road types, speed limits and slopes. Higher number = less frequent changes, lower number = more frequent changes.
randomness = 0.6
#Deceleration is initialised at 0 as the car does not start decelerating until it is over the speed limit.
deceleration = 0
#Acceleration starts at 0 because the car is not moving. This is similar to car speed.
acceleration = 0.0
#Distance tracks how far the car has moved.
distance = 0
#Time elapsed tracks how long the car has been driving.
timeElapsed = 0
#Slopes will affect the acceleration/deceleration of the car. We assume the car starts on flat ground.
slope = 0

#The acceleration function will accelerate the car. It takes speed limit, car speed and slope as parameters to calculate how much the car should accelerate.
def acceleration(limit, speed, slope):
    #First we find the difference between the speed limit and the car's current speed.
    limitDiff = limit - speed

    #This if else statement checks if the limit difference it lower than 60. If it is then we set acceleration to the difference divided by 6.5 + the slope.
    #This generates a slower acceleration gain over time when the difference is small to ensure we dont speed up to fast as we approach the speed limit.
    #We must + slope to ensure we make up for speed lost/gained by the slope.
    #If the difference is more than 60, we divide the limit diff by 4 to give it a larger increase in acceleration as we have a larger gap to cover.
    if(limitDiff < 60):
        acceleration = math.ceil(limitDiff / 6.5 + slope)
    else:
        acceleration = math.ceil(limitDiff / 4 + slope)
    
    #This if statement checks if the car will accelerate over the speed limit, if it will we need to accelerate an amount equal to the limit difference to ensure we do not speed over the limit.
    if(acceleration > limitDiff):
        acceleration = limitDiff
    return acceleration

#The deceleration function is similar to the acceleration function however it slows down the car instead of speeding it up. It also takes the speed limit
def deceleration(limit, speed, slope):
    #Calculating the difference in speed
    limitDiff = speed - limit
    #Turns slope into a positive integer.
    if(slope < 0):
        slope *= -1
    #This if else statement checks if the limit difference it higher than 15. If it is then we set deceleration to the difference multipled by 4 - the slope.
    #We must - slope to ensure we make up for speed lost/gained by the slope.
    #If the difference is less than 15, we multiply the limit diff by 4 to give it a smaller decrease in speed as we have a smaller gap to cover.
    if(limitDiff > 15):
        deceleration = math.ceil(limitDiff * 4 - slope)
    else:
        deceleration = math.ceil(limitDiff * 6.5 - slope)
    #If the deceleration is a negative integer, the car would accelerate which is not what we need. This acts as a failsafe.
    if(deceleration < 0):
        deceleration *= -1
    #If the deceleration is greater than the difference in speed, we should decrease deceleration to match the difference so we do not slow down too much.
    if(deceleration > limitDiff):
        deceleration = limitDiff
    return deceleration

#This function is used to create variations in road type, speed limit and slope.
def randomize(currentLimit, currentRoad, slope):
    #We get a random number from the imported library random.
    if(random.random() > randomness):
        #If it is higher than the randomness variable we set a random speed limit.
        currentLimit = random.choice(speedLimits)
    if(random.random() > randomness):
        #If it is higher than the randomness variable we set a random road type.
        currentRoad = roadType[random.randint(0,2)]
    if(random.random() > randomness):
        #If it is higher than the randomness variable we choose a random increase/decrease in slope. The max the slope can change by is 15 degrees.
        slope += random.randrange(-15,15)
        #This ensures slope is not greater than 45 degrees.
        if(slope > 45):
            slope = 45
        elif(slope < -45):
            slope = -45
    return currentLimit, currentRoad, slope

#This class is defined to hold road types and their deceleration.
class road:
    #Constructor
    def __init__(self, roadType, roadDecel):
        self.roadType = roadType
        self.roadDecel = roadDecel

#Creating three types of roads with different deceleration values.
roadType.append(road("City", 5))
roadType.append(road("Suburb", 10))
roadType.append(road("Dirt", 15))

#Setting the current road to a random road.
currentRoad = roadType[random.randint(0,len(roadType)-1)]
print()

#These two variables control if the car has cruise control on and if the brakes are applied. If the brakes are applied the cruise control should turn off.
cruise = True
brakes = False

#While brakes are not applied and cruise control is turned on.
while(brakes == False and cruise == True):
    #Sleep for 2 seconds, this ensures the program only updates every 2 seconds instead of every tick. Also add 2 seconds to the time elapsed counter so we can keep track of how long cruise control has been active for.
    time.sleep(2)
    timeElapsed += 2
    #Decelerate the car by the road deceleration rate.
    carSpeed -= currentRoad.roadDecel
    
    #Testing console outputs
    print("Speed after road decel")
    print(carSpeed)
    
    #Adjust car speed due to slope.
    carSpeed -= slope
    
    #More testing console outputs
    print("Speed after slope")
    print(carSpeed)
    print("Slope")
    print(slope)
    print("Speed before control")
    print(carSpeed)
    
    #If brakes are applied, turn off cruise control and break the loop.
    if(brakes == True):
        cruise = False
        break;
    
    #Adjust for deceleration
    carSpeed += currentRoad.roadDecel
    
    #Run the randomize function, which has a chance to randomize speed limit, current road and slope.
    currentLimit, currentRoad, slope = randomize(currentLimit, currentRoad, slope)
    
    #If the car is too fast, we need to accelerate, otherwise we need to decelerate.
    if(carSpeed < currentLimit):    
        carSpeed += acceleration(currentLimit, carSpeed, slope)
    if(carSpeed > currentLimit):
        carSpeed -= deceleration(currentLimit, carSpeed, slope)
    
    #Calculate the distance traveled. This is calculated as the car speed which is in km/h converted into m/s and then multiplied by 2 because each loop takes 2 seconds.
    distance += math.ceil(carSpeed * 0.277778 * 2)
    
    #More testing console outputs
    print("Speed after control:")
    print(carSpeed)
    print("Limit:")
    print(currentLimit)
    print("Road Type:")
    print(currentRoad.roadType)
    print("Distance Travelled:")
    print(distance)
    print("Time Elapsed:")
    print(timeElapsed)
    print("=========================")
    
    



Speed after road decel
-10
Speed after slope
-10
Slope
0
Speed before control
-10
Speed after control:
30
Limit:
100
Road Type:
Suburb
Distance Travelled:
17
Time Elapsed:
2
Speed after road decel
20
Speed after slope
15
Slope
5
Speed before control
15
Speed after control:
49
Limit:
70
Road Type:
Suburb
Distance Travelled:
45
Time Elapsed:
4
Speed after road decel
39
Speed after slope
22
Slope
17
Speed before control
22
Speed after control:
55
Limit:
70
Road Type:
Suburb
Distance Travelled:
76
Time Elapsed:
6
Speed after road decel
45
Speed after slope
28
Slope
17
Speed before control
28
Speed after control:
59
Limit:
60
Road Type:
Suburb
Distance Travelled:
109
Time Elapsed:
8
Speed after road decel
49
Speed after slope
32
Slope
17
Speed before control
32
Speed after control:
60
Limit:
60
Road Type:
Suburb
Distance Travelled:
143
Time Elapsed:
10
Speed after road decel
50
Speed after slope
33
Slope
17
Speed before control
33
Speed after control:
60
Limit:
60
Road Type:
Suburb
Distanc

KeyboardInterrupt: 